#  Display options:

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_BB"

print("--")


--


In [ ]:

#  Create database

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [4]:

#  Create graph
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph_BB, 5gzs2owKPao8mVVZJvg5AVtWQNfEpfEGVCHrcNV8RQHL, 0>


In [4]:

#  Connect to graph

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph_BB, 7JfcadZu8CmgPwwCfzQ92FXjh7QZYsHRToFJDtLSgPMd, 2>


In [5]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


Database Name: my_db                          Id: 6RXJt4q5o3fjZJcZZJpnBvC3gGYg7cy4CgJuqov2fMuC
   Graph Name: my_graph_BB                       Version: 0          Id: 5gzs2owKPao8mVVZJvg5AVtWQNfEpfEGVCHrcNV8RQHL
   Graph Name: my_graph_BC                       Version: 0          Id: HJcTkV737Vc4aN7T7w1nSntnb5Ln4zqtYfHJ1HJhCTFp
Database Name: default                        Id: 6k4ACPaga7Ze1hRhrcEziAGmP2F13rr48BYoakSFTGHZ

--


#  UDFs, Part 01: Just MPI

In [ ]:

#  Best practice; Have a second NoteBook open with the Operations Widget ready-
#
#  Why ?
#     When you make mistakes below, you will want to kill the Python job that is running
#     on the worker nodes, the UDFs you submit below-
#
#  Source,
#     https://mpi4py.readthedocs.io/en/stable/tutorial.html


In [ ]:

#  Example for testing; generates a random dataset
#

import time
   #
from IPython.display import clear_output


def my_func():
    
   from katana.distributed import single_host
      #
   import numpy as np                                                     #  Unparitioned objects are just fine; we are working from only
                                                                          #  one node
   l_return = np.random.randint(1, 101, 4)                                #  Generate an array[4] of random numbers in the range of 1-100

   return single_host(host=0, result=l_return)                            #  Multiple worker nodes (hosts), can only return from 1


l_cntr = 0
   #
while True:
   l_cntr += 1
   print("Running interation:  %-16d" % (l_cntr))
      #
   my_return = my_graph.run(lambda g: my_func())
   time.sleep(300) 
   clear_output(wait = True)
    


Running interation:  88              


          0/? [?op/s]